In [56]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

## prépare un mapping de l'index des circonscriptions de 2024 pour l'aligner avec celui de 2022

In [2]:
mapping_dept = (
    pd
    .read_csv('lg2022_circonscriptions.csv')
    .groupby(['CodDpt3Car', 'CodDpt'])
    .agg({'CodCirLg': len})
    .reset_index()
    .set_index('CodDpt3Car')
    [['CodDpt']]
    .query('CodDpt.str.startswith("Z")')
    .to_dict()
    ['CodDpt']
)

mapping_dept

{'099': 'ZZ',
 '971': 'ZA',
 '972': 'ZB',
 '973': 'ZC',
 '974': 'ZD',
 '975': 'ZS',
 '976': 'ZM',
 '977': 'ZX',
 '986': 'ZW',
 '987': 'ZP',
 '988': 'ZN'}

In [3]:
# candidats_dgfr = (
#     pd
#     .read_csv('https://www.data.gouv.fr/fr/datasets/r/9efe7b76-8257-4db5-9e9f-37abb81ce65d')
#     .assign(
#         NumCirc = lambda df: df['Code circonscription'].str[-2:].str.pad(3, fillchar="0"),
#         Dept = lambda df: df['Code circonscription'].str[:-2].str.pad(2, fillchar="0").str.pad(2, fillchar="0"),
#         CodCirc = lambda df: df.Dept.replace(mapping_dept) + df.NumCirc
#     )
# )

# candidats_dgfr

## récupère la liste des circonscriptions

In [4]:
base = 'https://www.resultats-elections.interieur.gouv.fr/telechargements/LG2024'

In [19]:
territoire_xml = requests.get(f'{base}/territoires/territoires.xml').text #.encode('latin-1')

In [5]:
territoires_url = f'{base}/territoires/territoires.xml'

In [24]:
regions = (
    pd
    .read_xml(
        StringIO(territoire_xml),
        xpath = '//Region',
        dtype="object"
     )
)

regions

,CodReg,LibReg,Departements
0,00,Non renseigné,\n
1,01,Guadeloupe,\n
2,02,Martinique,\n
3,03,Guyane,\n
4,04,La Réunion,\n
5,06,Mayotte,\n
6,11,Île-de-France,\n
7,24,Centre-Val de Loire,\n
8,27,Bourgogne-Franche-Comté,\n
9,28,Normandie,\n


In [27]:
def flat_codes(CodReg):
    departements = (
        (
            pd
            .read_xml(
                StringIO(territoire_xml),
                xpath = f'.//Region[./CodReg = "{CodReg}"]//Departement',
                dtype="object"
            )
            .assign(
                CodReg = CodReg
            )
        )
    )

    
    circonscriptions = (
        pd.concat([
            pd
            .read_xml(
                StringIO(territoire_xml),
                xpath = f'.//Departement[./CodDpt = "{CodDpt}"]//Circonscription',
                dtype="object"
            )
            .assign(
                CodReg = CodReg,
                CodDpt = CodDpt,
            )
            for CodDpt in departements.CodDpt.to_list()
        ])
        .assign(
            CodDpt = lambda df: df.CodCirElec.str[:-2].replace(mapping_dept)
        )
        .set_index('CodCirElec')
        .drop(['Communes'], axis=1)
    )
    
    return circonscriptions

circonscriptions = pd.concat([ flat_codes(CodReg) for CodReg in regions.CodReg.to_list() ])

circonscriptions

,LibCirElec,NbSap,CodReg,CodDpt
CodCirElec,,,,
97501,Saint-Pierre-et-Miquelon,1,00,975
98601,1ère circonscription,1,00,986
98701,1ere circonscription,1,00,987
98702,2ème circonscription,1,00,987
98703,3ème circonscription,1,00,987
...,...,...,...,...
8405,5ème circonscription,1,93,84
2A01,1ère circonscription,1,94,2A
2A02,2ème circonscription,1,94,2A


In [20]:
# circonscriptions = (
#     pd
#     .read_xml(
#         territoire_xml,
#         xpath='.//Circonscription',
#         dtype="object"
#     )
#     .assign(
#         CodDpt = lambda df: df.CodCirElec.str[:-2].replace(mapping_dept)
#     )
#     .set_index('CodCirElec')
#     .drop(['Communes'], axis=1)
# )

# circonscriptions

In [7]:
circonscriptions.to_csv('lg2024_circonscriptions.csv')

## récupère la liste des candidat·e·s du premier tour

In [174]:
def get_candidats_t1(circonscription):
    dept = circonscription[:-2]
    circ = circonscription

    url = f'https://www.resultats-elections.interieur.gouv.fr/telechargements/LG2024/candidatsT1/{dept}/C1{dept}{circonscription}.xml'
        
    try:
        df = (
            pd
            .read_xml(
                url,
                xpath='.//Candidat'
            )
            .assign(CodCirc=circonscription)
        )
    except:
        print(f'error: {circonscription}')
        df = None
        
    
    return df
    
    
candidats_2024_t1 = pd.concat(
    [
        get_candidats_t1(circonscription)
        for circonscription in circonscriptions.index.to_list()
    ]
)

candidats_2024_t1

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,CodCirc
0,1,LENORMAND,Stéphane,M.,DVD,Divers droite,97501
1,2,BEAUMONT,Frédéric,M.,SOC,Parti socialiste,97501
2,3,CHAGNON,Patricia,Mme,RN,Rassemblement National,97501
3,4,LETOURNEL,Marion,Mme,FI,La France insoumise,97501
4,5,LEBAILLY,Patrick,M.,DVG,Divers gauche,97501
...,...,...,...,...,...,...,...
3,4,RONGIONE,Viviane,Mme,EXG,Extrême gauche,2B02
4,5,GIACOMI,Jean-Antoine,M.,REG,Régionaliste,2B02
5,6,JOUART,Sylvie,Mme,RN,Rassemblement National,2B02
6,7,CARLI,Antò,M.,REG,Régionaliste,2B02


In [175]:
candidats_2024_t1_2 = (
    candidats_2024_t1
    .assign(
        CodCirc2 = lambda df: df.CodCirc.str[:-2].replace(mapping_dept) + "0" + df.CodCirc.str[-2:]
    )
)

candidats_2024_t1_2

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,CodCirc,CodCirc2
0,1,LENORMAND,Stéphane,M.,DVD,Divers droite,97501,ZS001
1,2,BEAUMONT,Frédéric,M.,SOC,Parti socialiste,97501,ZS001
2,3,CHAGNON,Patricia,Mme,RN,Rassemblement National,97501,ZS001
3,4,LETOURNEL,Marion,Mme,FI,La France insoumise,97501,ZS001
4,5,LEBAILLY,Patrick,M.,DVG,Divers gauche,97501,ZS001
...,...,...,...,...,...,...,...,...
3,4,RONGIONE,Viviane,Mme,EXG,Extrême gauche,2B02,2B002
4,5,GIACOMI,Jean-Antoine,M.,REG,Régionaliste,2B02,2B002
5,6,JOUART,Sylvie,Mme,RN,Rassemblement National,2B02,2B002
6,7,CARLI,Antò,M.,REG,Régionaliste,2B02,2B002


In [176]:
candidats_2024_t1_2.to_csv('lg2024_t1_candidats.csv', index=False)

### transpose la liste des candidat·e·s dans un format *wide*

In [177]:
candidats_2024_t1_wide = (
    candidats_2024_t1
    .assign(
        Nom_Prenom = lambda df: df.NomPsn + " " +df.PrenomPsn
    )
    .pivot_table(
        index = 'CodCirc',
        columns = 'CodNuaCand',
        values = 'Nom_Prenom',
        aggfunc= lambda x : ', '.join(x)
    )
    .replace(np.nan, '')
)

candidats_2024_t1_wide

CodNuaCand,COM,DIV,DSV,DVC,DVD,DVG,ECO,ENS,EXD,EXG,...,LR,RDG,REC,REG,RN,SOC,UDI,UG,UXD,VEC
CodCirc,,,,,,,,,,,,,,,,,,,,,
0101,,,"VINCENT Cyril, MENDES Michael",,,,,GUILLERMIN Vincent,,LAHY Éric,...,BRETON Xavier,,,,MAÎTRE Christophe,,,GUERAUD Sébastien,,
0102,,,,,,,,DAUBIÉ Romain,EYRAUD Olivier,GOUTAGNY Vincent,...,NANCHI Alexandre,,,,KOTARAC Andréa,,,MEYER Maxime,,
0103,,"TONIZZO Sofia, KOUASSI Fulgence",VEILLEROT Annick,,,,,GIVERNET Olga,,MAISONNETTE Cécile,...,UNAL Khadija,,,,DUBARRY Karine,,,JOLIE Christian,,
0104,,"BRESSON Yannick, NICAUD Jérémy",,,,,,COQUELET Christophe,,COUSSON Sylvain,...,BILLOUDET Guy,,,,BUISSON Jérôme,,,LIOTIER Charline,,
0105,,,,,ABAD Damien,,CHATELARD Thomas,DESCOURS Nathalie,,CROZET Sylvie,...,BOURDIN Fabrice,,PATRU Maria Cristina,,,,,PISANI Florence,CHAVENT Marc,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ07,,"GEFFRAY Fanny, CHAMBON Jérôme",,HUQUET Isabelle,,RICHARD Cécile,,PETIT Frédéric,,,...,MIER-GARRIGOU Dominique,,ALEXANDRE Julie,,NAVEYS--DUMAS Mathilde,,,RHARMAOUI-CLAQUIN Asma,,
ZZ08,,"SPITALAS Nicolas, BIZET David, SIGOURA Benjamin",,NEFFATI Gilles,HABABOU SOLOMON Philippe,CHARTRAIN Valérie,,YADAN Caroline,,,...,"HABIB Meyer, ASSOULINE Aurelie",,BENSOUSSAN Guillaume,,,,,LERER Yaël,,
ZZ09,,"OUDRHIRI Hassan, SIDIBÉ Gabriel Marie, DIANIFA...",FADILI Hachim,"TAHIRI Rachid, DUCELLIER Régina","SACKHO Kourtoum, DAVOUX Erwan Borhan","BOUDJEKADA Ismaël, TINAUGUS Edouard, KHALFI Se...",,DJOUADI Samira,,,...,BADREDDINE Jihad,,DREVON Pierre,,CHARRON Elodie,,,BEN CHEÏKH Karim,,


In [178]:
candidats_2024_t1_wide.to_csv('lg2024_t1_candidats.wide.csv')

## précalcule un tableau sur la présence des nuances dans chacune des circonscriptions

In [179]:
nuances2024 = (
    candidats_2024_t1
    .pivot_table(
        index = 'CodCirc',
        columns = 'CodNuaCand',
        values = 'NomPsn',
        aggfunc= lambda x : len(x) >= 1
    )
    .fillna(False)
)

nuances2024

CodNuaCand,COM,DIV,DSV,DVC,DVD,DVG,ECO,ENS,EXD,EXG,...,LR,RDG,REC,REG,RN,SOC,UDI,UG,UXD,VEC
CodCirc,,,,,,,,,,,,,,,,,,,,,
0101,False,False,True,False,False,False,False,True,False,True,...,True,False,False,False,True,False,False,True,False,False
0102,False,False,False,False,False,False,False,True,True,True,...,True,False,False,False,True,False,False,True,False,False
0103,False,True,True,False,False,False,False,True,False,True,...,True,False,False,False,True,False,False,True,False,False
0104,False,True,False,False,False,False,False,True,False,True,...,True,False,False,False,True,False,False,True,False,False
0105,False,False,False,False,True,False,True,True,False,True,...,True,False,True,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ07,False,True,False,True,False,True,False,True,False,False,...,True,False,True,False,True,False,False,True,False,False
ZZ08,False,True,False,True,True,True,False,True,False,False,...,True,False,True,False,False,False,False,True,False,False
ZZ09,False,True,True,True,True,True,False,True,False,False,...,True,False,True,False,True,False,False,True,False,False


In [180]:
nuances2024.to_csv('lg2024_t1_nuances.csv')

## récupère la liste des résultats du premier tour

In [534]:
def get_resultats_t1(CodCirc, CodDept, CodeReg):
    url = f'{base}/resultatsT1/{CodDept}/R1{CodCirc}.xml'
    
    try:
        df = (
            pd
            .read_xml(
                url,
                xpath = './/Candidat',
                dtype = 'object'
            )
            .assign(
                CodCirc=CodCirc,
                CodDept=CodDept,
                CodeReg=CodeReg
            )
        )
        #print(url)
    except:
        #print(f'error: {circonscription}')
        df = None
        
    
    return df
    
    
resultats_t1 = pd.concat(
    [
        get_resultats_t1(c['CodCirElec'], c['CodDpt'], c['CodReg'])
        for idx, c in tqdm(list(circonscriptions.reset_index().iterrows()))
    ]
)
resultats_t1

  0%|          | 0/577 [00:00<?, ?it/s]

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,CodCirc,CodDept,CodeReg
0,1,LENORMAND,Stéphane,M.,DVD,Divers droite,1184,"43,09","23,36",QUALIF T2,97501,975,00
1,2,BEAUMONT,Frédéric,M.,SOC,Parti socialiste,464,"16,89","9,15",QUALIF T2,97501,975,00
2,3,CHAGNON,Patricia,Mme,RN,Rassemblement National,291,"10,59","5,74",NON,97501,975,00
3,4,LETOURNEL,Marion,Mme,FI,La France insoumise,409,"14,88","8,07",NON,97501,975,00
4,5,LEBAILLY,Patrick,M.,DVG,Divers gauche,400,"14,56","7,89",NON,97501,975,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,4,RONGIONE,Viviane,Mme,EXG,Extrême gauche,280,"0,63","0,41",NON,2B02,2B,94
4,5,GIACOMI,Jean-Antoine,M.,REG,Régionaliste,0,"0,00","0,00",NON,2B02,2B,94
5,6,JOUART,Sylvie,Mme,RN,Rassemblement National,11275,"25,42","16,60",QUALIF T2,2B02,2B,94
6,7,CARLI,Antò,M.,REG,Régionaliste,2277,"5,13","3,35",NON,2B02,2B,94


In [535]:
resultats_t1_2 = (
    resultats_t1
    .join(
        circonscriptions
        .reset_index()
        .assign(
            CodCirc2 = lambda df: (
                df.CodDpt.replace(mapping_dept)
                + df.CodCirElec.astype(str).str[-2:].str.pad(3, fillchar='0')
            )
        )
        .set_index('CodCirElec'), #[['CodCirc2']],
        on = 'CodCirc'
    )
)

resultats_t1_2

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,CodCirc,CodDept,CodeReg,LibCirElec,NbSap,CodReg,CodDpt,CodCirc2
0,1,LENORMAND,Stéphane,M.,DVD,Divers droite,1184,"43,09","23,36",QUALIF T2,97501,975,00,Saint-Pierre-et-Miquelon,1,00,975,ZS001
1,2,BEAUMONT,Frédéric,M.,SOC,Parti socialiste,464,"16,89","9,15",QUALIF T2,97501,975,00,Saint-Pierre-et-Miquelon,1,00,975,ZS001
2,3,CHAGNON,Patricia,Mme,RN,Rassemblement National,291,"10,59","5,74",NON,97501,975,00,Saint-Pierre-et-Miquelon,1,00,975,ZS001
3,4,LETOURNEL,Marion,Mme,FI,La France insoumise,409,"14,88","8,07",NON,97501,975,00,Saint-Pierre-et-Miquelon,1,00,975,ZS001
4,5,LEBAILLY,Patrick,M.,DVG,Divers gauche,400,"14,56","7,89",NON,97501,975,00,Saint-Pierre-et-Miquelon,1,00,975,ZS001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,4,RONGIONE,Viviane,Mme,EXG,Extrême gauche,280,"0,63","0,41",NON,2B02,2B,94,2ème circonscription,1,94,2B,2B002
4,5,GIACOMI,Jean-Antoine,M.,REG,Régionaliste,0,"0,00","0,00",NON,2B02,2B,94,2ème circonscription,1,94,2B,2B002
5,6,JOUART,Sylvie,Mme,RN,Rassemblement National,11275,"25,42","16,60",QUALIF T2,2B02,2B,94,2ème circonscription,1,94,2B,2B002
6,7,CARLI,Antò,M.,REG,Régionaliste,2277,"5,13","3,35",NON,2B02,2B,94,2ème circonscription,1,94,2B,2B002


In [536]:
len(resultats_t1_2.CodCirc.unique())

577

In [537]:
circonscriptions.query('~CodCirElec.isin(@resultats_t1_2.CodCirc)')

,LibCirElec,NbSap,CodReg,CodDpt
CodCirElec,,,,


### élu·e·s au premier tour

In [538]:
resultats_t1_2.query('Elu == "OUI"')

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,CodCirc,CodDept,CodeReg,LibCirElec,NbSap,CodReg,CodDpt,CodCirc2
3,4,SEO,Mikaele,M.,ENS,Ensemble ! (Majorité présidentielle),4282,"62,23","47,41",OUI,98601,986,00,1ère circonscription,1,00,986,ZW001
2,3,FREBAULT,Moerani,M.,DVD,Divers droite,18456,"53,85","25,02",OUI,98701,987,00,1ere circonscription,1,00,987,ZP001
1,2,YOUSSOUFFA,Estelle,Mme,DVD,Divers droite,13640,"79,48","29,87",OUI,97601,976,06,1ère circonscription,1,06,976,ZM001
10,11,AMIRSHAHI,Pouria,M.,UG,Union de la gauche,32152,"54,27","40,47",OUI,7505,75,11,5ème circonscription,1,11,75,75005
0,1,CHIKIROU,Sophia,Mme,UG,Union de la gauche,34329,"58,19","42,26",OUI,7506,75,11,6ème circonscription,1,11,75,75006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,4,LOTTIAUX,Philippe,M.,RN,Rassemblement National,40631,"56,11","35,66",OUI,8304,83,93,4ème circonscription,1,93,83,83004
6,7,LECHANTEUX,Julie,Mme,RN,Rassemblement National,34496,"51,51","33,70",OUI,8305,83,93,5ème circonscription,1,93,83,83005
0,1,GILETTI,Frank,M.,RN,Rassemblement National,45882,"53,51","35,61",OUI,8306,83,93,6ème circonscription,1,93,83,83006
3,4,SCHRECK,Philippe,M.,RN,Rassemblement National,38433,"53,55","34,89",OUI,8308,83,93,8ème circonscription,1,93,83,83008


In [539]:
len(resultats_t1_2.query('Elu == "OUI"'))

76

### RN

In [540]:
len(resultats_t1_2.query('Elu == "OUI" and (CodNuaCand == "RN" or CodNuaCand == "UXD")'))

38

In [541]:
len(resultats_t1_2.query('Elu == "QUALIF T2" and (CodNuaCand == "RN" or CodNuaCand == "UXD")'))

444

In [542]:
resultats_t1_2.query('(CodNuaCand == "RN" or CodNuaCand == "UXD")').NbVoix.astype(int).sum()

10621385

In [543]:
resultats_t1_2.query('(CodNuaCand == "RN" or CodNuaCand == "UXD")').NbVoix.astype(int).sum() / resultats_t1_2.NbVoix.astype(int).sum()

0.3313564353491996

### NFP

In [544]:
len(resultats_t1_2.query('Elu == "OUI" and CodNuaCand == "UG"'))

32

In [545]:
len(resultats_t1_2.query('Elu == "QUALIF T2" and CodNuaCand == "UG"'))

407

### triangulaires

In [546]:
resultats_t1_2_triangulaire = (
    resultats_t1_2
    .query('Elu == "QUALIF T2"')
    .groupby('CodCirc2')
    .agg({
        'NomPsn': 'count'
    })
    .assign(
        triangulaire= lambda df: df.NomPsn > 2
    )
    .query('triangulaire')
)

resultats_t1_2_triangulaire

,NomPsn,triangulaire
CodCirc2,,
01001,3,True
01002,3,True
01003,3,True
01004,3,True
03002,3,True
...,...,...
95002,3,True
95003,3,True
95004,3,True


In [547]:
len(resultats_t1_2_triangulaire)

311

In [548]:
# Nouveau Front populaire (UG): #FC392B
# Divers gauche (EXG, COM, FI, SOC, RDG, VEC, DVG, ECO): #D94B4B
# Ensemble (ENS, REN, MDM, HOR, UDI): #F0B500
# Divers centre (DVC): #ff8800
# Les Républicains (LR): #4292c6
# Divers droite (DVD, REC, DSV, EXD): #08306b
# Rassemblement national (RN): #662506
# Rassemblement national-Les Républicains (UXD): #cc4c02
# Autres (REG, DIV): #949191

nuance_to_groupe = {
    'UG' : 'Nouveau Front populaire',
    'EXG': 'Divers gauche',
    'COM': 'Divers gauche',
    'FI' : 'Divers gauche',
    'SOC': 'Divers gauche',
    'RDG': 'Divers gauche', 
    'VEC': 'Divers gauche', 
    'DVG': 'Divers gauche',
    'ECO': 'Divers gauche',
    'ENS': 'Ensemble',
    'REN': 'Ensemble',
    'MDM': 'Ensemble',
    'HOR': 'Ensemble',
    'UDI': 'Ensemble',
    'DVC': 'Divers centre',
    'LR' : 'Les Républicains',
    'DVD': 'Divers droite',
    'REC': 'Divers droite',
    'DSV': 'Divers droite',
    'EXD': 'Divers droite',
    'RN' : 'Rassemblement national',
    'UXD': 'Rassemblement national-Les Républicains',
    'REG': 'Autres',
    'DIV': 'Autres'
    
}

t1_entete = (
    resultats_t1_2
    .assign(
        NbVoix = lambda df: df.NbVoix.astype(int)
    )
    .sort_values('NbVoix', ascending=False)
    .drop_duplicates('CodCirc2', keep='first')
    .set_index('CodCirc2')
    .sort_index()
    .assign(
        GrpLib = lambda df: df.CodNuaCand.apply(lambda x: nuance_to_groupe[x]),
        CandLib = lambda df: df.PrenomPsn+" "+df.NomPsn
    )
)

t1_entete

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,CodCirc,CodDept,CodeReg,LibCirElec,NbSap,CodReg,CodDpt,GrpLib,CandLib
CodCirc2,,,,,,,,,,,,,,,,,,,
01001,2,MAÎTRE,Christophe,M.,RN,Rassemblement National,23819,"39,37","27,43",QUALIF T2,0101,01,84,1ère circonscription,1,84,01,Rassemblement national,Christophe MAÎTRE
01002,1,KOTARAC,Andréa,M.,RN,Rassemblement National,28189,"39,20","27,67",QUALIF T2,0102,01,84,2ème circonscription,1,84,01,Rassemblement national,Andréa KOTARAC
01003,1,GIVERNET,Olga,Mme,ENS,Ensemble ! (Majorité présidentielle),17420,"32,43","20,71",QUALIF T2,0103,01,84,3ème circonscription,1,84,01,Ensemble,Olga GIVERNET
01004,6,BUISSON,Jérôme,M.,RN,Rassemblement National,30221,"46,01","31,44",QUALIF T2,0104,01,84,4ème circonscription,1,84,01,Rassemblement national,Jérôme BUISSON
01005,8,CHAVENT,Marc,M.,UXD,Union de l'extrême droite,20161,"39,12","25,88",QUALIF T2,0105,01,84,5ème circonscription,1,84,01,Rassemblement national-Les Républicains,Marc CHAVENT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ007,1,PETIT,Frédéric,M.,ENS,Ensemble ! (Majorité présidentielle),21929,"37,78","16,76",QUALIF T2,ZZ07,ZZ,00,7ème circonscription,1,00,ZZ,Ensemble,Frédéric PETIT
ZZ008,5,HABIB,Meyer,M.,LR,Les Républicains,11557,"35,58","7,76",QUALIF T2,ZZ08,ZZ,00,8ème circonscription,1,00,ZZ,Les Républicains,Meyer HABIB
ZZ009,9,BEN CHEÏKH,Karim,M.,UG,Union de la gauche,15041,"49,35","13,40",NON,ZZ09,ZZ,00,9ème circonscription,1,00,ZZ,Nouveau Front populaire,Karim BEN CHEÏKH


In [549]:
t1_entete.query('(CodNuaCand == "RN" or CodNuaCand == "UXD")')

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,CodCirc,CodDept,CodeReg,LibCirElec,NbSap,CodReg,CodDpt,GrpLib,CandLib
CodCirc2,,,,,,,,,,,,,,,,,,,
01001,2,MAÎTRE,Christophe,M.,RN,Rassemblement National,23819,"39,37","27,43",QUALIF T2,0101,01,84,1ère circonscription,1,84,01,Rassemblement national,Christophe MAÎTRE
01002,1,KOTARAC,Andréa,M.,RN,Rassemblement National,28189,"39,20","27,67",QUALIF T2,0102,01,84,2ème circonscription,1,84,01,Rassemblement national,Andréa KOTARAC
01004,6,BUISSON,Jérôme,M.,RN,Rassemblement National,30221,"46,01","31,44",QUALIF T2,0104,01,84,4ème circonscription,1,84,01,Rassemblement national,Jérôme BUISSON
01005,8,CHAVENT,Marc,M.,UXD,Union de l'extrême droite,20161,"39,12","25,88",QUALIF T2,0105,01,84,5ème circonscription,1,84,01,Rassemblement national-Les Républicains,Marc CHAVENT
02001,4,DRAGON,Nicolas,M.,RN,Rassemblement National,24774,"54,49","34,38",OUI,0201,02,32,1ère circonscription,1,32,02,Rassemblement national,Nicolas DRAGON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91002,6,DA CONCEICAO CARVALHO,Nathalie,Mme,RN,Rassemblement National,24608,"40,30","26,48",QUALIF T2,9102,91,11,2ème circonscription,1,11,91,Rassemblement national,Nathalie DA CONCEICAO CARVALHO
91003,5,MILOSEVIC,Stefan,M.,RN,Rassemblement National,22290,"33,01","22,48",QUALIF T2,9103,91,11,3ème circonscription,1,11,91,Rassemblement national,Stefan MILOSEVIC
95001,6,SICARD,Anne,Mme,RN,Rassemblement National,18823,"33,65","22,40",QUALIF T2,9501,95,11,1ère circonscription,1,11,95,Rassemblement national,Anne SICARD


In [550]:
lg2022_resultats = (
    pd
    .read_csv('lg2022_t2_resultats.csv')
    .assign(
        NbVoix = lambda df: df.NbVoix.astype(int)
    )
    .sort_values('NbVoix', ascending=False)
    .drop_duplicates('CodCirc2', keep='first')
    .pipe(lambda df: 
        pd.concat([
            df,
            pd
            .read_csv('lg2022_t1_resultats.csv')
            .query('Elu == "oui"')
        ])
    )
    .set_index('CodCirc2')
    .sort_index()
)

lg2022_resultats

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNua,LibNua,NbVoix,RapportExprime,RapportInscrit,Elu,CodCirc,CodCirc1
CodCirc2,,,,,,,,,,,,
01001,3,BRETON,Xavier,M.,LR,Les Républicains,24408,"63,22","28,32",oui,00101,00101
01002,2,DAUBIÉ,Romain,M.,ENS,Ensemble ! (Majorité présidentielle),24960,"58,34","24,97",oui,00102,00102
01003,3,GIVERNET,Olga,Mme,ENS,Ensemble ! (Majorité présidentielle),18398,"58,72","22,38",oui,00103,00103
01004,4,BUISSON,Jérôme,M.,RN,Rassemblement National,22601,"62,27","23,81",oui,00104,00104
01005,4,ABAD,Damien,M.,DVD,Divers droite,17687,"57,86","22,83",oui,00105,00105
...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ007,2,PETIT,Frédéric,M.,ENS,Ensemble ! (Majorité présidentielle),23191,"60,21","18,79",oui,09907,09907
ZZ008,8,HABIB,Meyer,M.,UDI,Union des Démocrates et des Indépendants,8470,"50,58","6,46",oui,09908,09908
ZZ009,3,BEN CHEÏKH,Karim,M.,NUP,Nouvelle union populaire écologique et sociale,11348,"54,07","9,41",oui,09909,09909


In [551]:
t1_export = (
    circonscriptions
    .reset_index()
    .assign(
        CodCirc2 = lambda df: (
            df.CodDpt.replace(mapping_dept)
            + df.CodCirElec.astype(str).str[-2:].str.pad(3, fillchar='0')
        )
    )
    .set_index('CodCirc2')[[]]
    .sort_index()
    .join(
        t1_entete[['CandLib', 'CodNuaCand', 'LibNuaCand', 'GrpLib', 'Elu']]
    )
    .assign(
        Elu = lambda df: df.Elu.replace({ "QUALIF T2": "", "NON": "" })
    )
    .join(
        lg2022_resultats
        .assign(
            SortantLabel = lambda df: df.PrenomPsn+" "+df.NomPsn+" - "+df.LibNua
        )
        [['SortantLabel']]
    )
    .fillna('')
)

t1_export

,CandLib,CodNuaCand,LibNuaCand,GrpLib,Elu,SortantLabel
CodCirc2,,,,,,
01001,Christophe MAÎTRE,RN,Rassemblement National,Rassemblement national,,Xavier BRETON - Les Républicains
01002,Andréa KOTARAC,RN,Rassemblement National,Rassemblement national,,Romain DAUBIÉ - Ensemble ! (Majorité président...
01003,Olga GIVERNET,ENS,Ensemble ! (Majorité présidentielle),Ensemble,,Olga GIVERNET - Ensemble ! (Majorité président...
01004,Jérôme BUISSON,RN,Rassemblement National,Rassemblement national,,Jérôme BUISSON - Rassemblement National
01005,Marc CHAVENT,UXD,Union de l'extrême droite,Rassemblement national-Les Républicains,,Damien ABAD - Divers droite
...,...,...,...,...,...,...
ZZ007,Frédéric PETIT,ENS,Ensemble ! (Majorité présidentielle),Ensemble,,Frédéric PETIT - Ensemble ! (Majorité présiden...
ZZ008,Meyer HABIB,LR,Les Républicains,Les Républicains,,Meyer HABIB - Union des Démocrates et des Indé...
ZZ009,Karim BEN CHEÏKH,UG,Union de la gauche,Nouveau Front populaire,,Karim BEN CHEÏKH - Nouvelle union populaire éc...


In [552]:
t1_export.to_csv('lg2024_t1_flourish.csv')

## synthèse

In [553]:
t1_pos = (
    resultats_t1_2
    .assign(
        NbVoix = lambda df: df.NbVoix.astype(int)
    )
    .query("Elu == 'QUALIF T2'")
    .pivot_table(
        index   = "CodCirc2",
        columns = "LibNuaCand",
        values = "NbVoix",
        aggfunc = "sum"
    )
    .rank(axis=1, ascending=False)
    .fillna(0)
    .astype(int)
    .apply(pd.Series.value_counts)
    .T
    #.value_counts(axis=0)
    [[1,2,3,4]]
    .fillna(0)
    .astype(int)
)

t1_pos

,1,2,3,4
LibNuaCand,,,,
Divers,0,2,1,0
Divers centre,3,5,3,0
Divers droite,16,18,8,1
Divers gauche,12,4,3,0
Droite souverainiste,0,1,0,0
Ecologistes,1,0,0,0
Ensemble ! (Majorité présidentielle),59,144,89,0
Extrême droite,0,1,0,0
Horizons,3,5,3,0


In [554]:
(
    resultats_t1_2
    .pivot_table(
        index="LibNuaCand",
        columns="Elu",
        values="NomPsn",
        aggfunc= lambda x: len(x)
    )
    .fillna(0)
    .astype(int)
    .sort_values('QUALIF T2', ascending=False)
    .join(t1_pos)
    .join(
        resultats_t1_2
        .assign(
            NbVoix = lambda df: df.NbVoix.astype(int)
        )
        .groupby('LibNuaCand')
        [['NbVoix']]
        .sum()
    )
    .fillna(0)
    .astype(int)
)

,NON,OUI,QUALIF T2,1,2,3,4,NbVoix
LibNuaCand,,,,,,,,
Union de la gauche,105,32,407,122,154,129,2,8968561
Rassemblement National,75,37,389,222,106,60,1,9370184
Ensemble ! (Majorité présidentielle),149,2,292,59,144,89,0,6421654
Union de l'extrême droite,7,1,55,36,12,7,0,1251201
Les Républicains,251,1,54,15,33,5,1,2104285
Divers droite,147,2,43,16,18,8,1,1177650
Divers gauche,120,0,20,12,4,3,0,490885
Régionaliste,115,0,14,9,4,0,0,335808
Horizons,5,0,11,3,5,3,0,231664
